In [1]:
import sys
sys.path.append('/cluster/sj1')

In [2]:
%load_ext autoreload
%autoreload 2

import torch
import pyro
import numpy as np
from scipy.stats import kendalltau
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import seaborn as sns
from bb_opt.src.bayesian_opt import (
    optimize,
    get_model_bnn,
    train_model_bnn,
    partial_train_model_bnn,
    train,
    bnn_predict
)
from bb_opt.src.utils import get_path, save_pyro_model, load_pyro_model
from bb_opt.src import hsic as hsic
from bb_opt.src import knn_mi as knn_mi
from gpu_utils.utils import gpu_init
from tqdm import tnrange

gpu_id = gpu_init()
print(f"Running on GPU {gpu_id}")
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

%matplotlib inline

/cluster/sj1/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


Running on GPU 0


In [4]:
import bb_opt.src.utils as utils

In [3]:
n_train = 1000

project = "dna_binding"
dataset = "crx_ref_r1"

exclude_top = 0.01

root = "/cluster/sj1/bb_opt/"
data_dir = get_path(root, "data", project, dataset)
inputs = np.load(get_path(data_dir, "inputs.npy"))
labels = np.load(get_path(data_dir, "labels.npy"))

idx = np.arange(labels.shape[0])

sort_idx = labels.argsort()[:-int(labels.shape[0]*exclude_top)]
idx = idx[sort_idx]

train_idx, _, _ = utils.train_val_test_split(idx, split=[n_train, 0])
train_idx2, _, test_idx2 = utils.train_val_test_split(n_train, split=[0.9, 0])

test_idx = train_idx[test_idx2]
train_idx = train_idx[train_idx2]

train_inputs = inputs[train_idx]
train_labels = inputs[train_idx]

test_inputs = inputs[test_idx]
test_labels = inputs[test_idx]

/cluster/sj1/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
